In [5]:
!rm -rf qiskit-optimization/
!git clone https://github.com/morim3/qiskit-optimization

Cloning into 'qiskit-optimization'...
remote: Enumerating objects: 4168, done.
remote: Counting objects: 100% (233/233), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 4168 (delta 146), reused 154 (delta 81), pack-reused 3935
Receiving objects: 100% (4168/4168), 5.08 MiB | 13.07 MiB/s, done.
Resolving deltas: 100% (2782/2782), done.


In [ ]:
!nvidia-smi
# !pip install -r qiskit-optimization/requirements-dev.txt
!pip install qiskit-aer-gpu
!pip install docplex
!pip install pylatexenc

import sys
sys.path.append("./qiskit-optimization/")

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 22.8 MB 85.9 MB/s 
     |████████████████████████████████| 4.8 MB 36.3 MB/s 
     |████████████████████████████████| 929 kB 54.4 MB/s 
     |████████████████████████████████| 37.5 MB 515 kB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
     |████████████████████████████████| 49 kB 217 kB/s 
     |████████████████████████████████| 1.9 MB 41.8 MB/s 
     |████████████████████████████████| 112 kB 60.8 MB/s 


In [ ]:
%load_ext autoreload
%autoreload

from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_optimization.algorithms import PQCGroverOptimizer, GroverOptimizer, MinimumEigenOptimizer
from qiskit_optimization.algorithms.pqc_grover_optimizer import ProposeDistribution
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.circuit.library import TwoLocal
from docplex.mp.model import Model
import numpy as np
print(Aer.backends())
backend = Aer.get_backend('aer_simulator')
backend.set_options(device='GPU')
backend_statevector = Aer.get_backend('statevector_simulator')
print(backend)

In [ ]:
import itertools
model = Model()
xs = [model.binary_var(name=f"x{i}") for i in range(15)]
objective = sum([np.random.randint(1, 4) * x for x in xs]) + \
     sum([np.random.randint(-4, -1) * x * y 
     for x, y in itertools.combinations(xs, 2) 
     if np.random.choice([0, 1], p=[2/3, 1/3])==1])
# objective = (-xs[0]+2*xs[1]-3*xs[2]-2*xs[0]*xs[2]-1*xs[1]*xs[2])
model.minimize(objective)
qp = from_docplex_mp(model)
print(qp.prettyprint())

In [185]:
exact_solver = MinimumEigenOptimizer(NumPyMinimumEigensolver())
exact_result = exact_solver.solve(qp)
print(exact_result.prettyprint())

objective function value: -86.0
variable values: x0=1.0, x1=1.0, x2=1.0, x3=1.0, x4=1.0, x5=1.0, x6=1.0, x7=1.0, x8=1.0, x9=1.0, x10=1.0, x11=1.0, x12=1.0, x13=1.0, x14=1.0
status: SUCCESS


In [4]:
num_key_qbits = len(qp.objective.linear.to_array())
init_state = QuantumCircuit(num_key_qbits)
init_state.h(range(num_key_qbits))
ansatz = TwoLocal(num_key_qbits, 'ry', 'crx', 'circular', reps=1,
                  insert_barriers=True,
                  initial_state=init_state,
                  skip_final_rotation_layer=True,
                  )
ansatz.decompose().draw("mpl")


NameError: ignored

In [ ]:
# from qiskit import execute, QuantumCircuit, transpile

# ansatz_bound = ansatz.assign_parameters(np.array([0.] * num_key_qbits+[np.arcsin(np.sqrt(0.5))*2]*num_key_qbits))

# qc = QuantumCircuit(num_key_qbits)
# # qc.h(range(num_key_qbits))
# qc.compose(ansatz_bound, inplace=True)

# qc = transpile(qc, backend=backend_statevector)
# result = backend_statevector.run(qc).result()
# result.get_statevector(qc)

In [1]:
print(ansatz.num_parameters)
init_param = np.zeros(ansatz.num_parameters)
propose_distribution = ProposeDistribution(
    ansatz,                 
    num_key_qbits, 
    QuantumInstance(backend=backend, shots=1024, seed_simulator=99),
    init_param=init_param
    )
grover_optimizer = PQCGroverOptimizer(
    num_value_qubits=8, 
    propose_dist=propose_distribution,
    num_iterations=15, 
    quantum_instance=backend,      
    sample_num=1, quantile=0., 
    positive_train_conf={"iter": 5, "learning_rate": 0.001},
    negative_train_conf={"iter": 5, "learning_rate": 0.001})
results = grover_optimizer.solve(qp)
print(results.prettyprint())
print(results.operation_counts)

NameError: ignored

In [ ]:
def eval_num(operation_dict):
    eval_num = 0
    for x in operation_dict.values():
        eval_num += x['Q(x)']
        if 'Q' in x:
            eval_num += x['Q']
    return eval_num

In [ ]:
eval_num(results.operation_counts)

91

In [ ]:
grover_optimizer = GroverOptimizer(8, num_iterations=15, quantum_instance=backend,)
results = grover_optimizer.solve(qp)
print(results.prettyprint())
print(results.operation_counts)

objective function value: -26.0
variable values: x0=1.0, x1=1.0, x2=1.0, x3=1.0, x4=1.0, x5=0.0, x6=1.0, x7=0.0, x8=1.0, x9=0.0, x10=0.0, x11=1.0, x12=0.0, x13=0.0, x14=1.0
status: SUCCESS
{0: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1)]), 1: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1)]), 2: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1)]), 3: OrderedDict([('measure', 24), ('h', 15), ('Q', 2), ('Q(x)', 1)]), 4: OrderedDict([('measure', 24), ('h', 15), ('Q', 3), ('Q(x)', 1)]), 5: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1)]), 6: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1), ('Q', 1)]), 7: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1)]), 8: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1), ('Q', 1)]), 9: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1)]), 10: OrderedDict([('measure', 24), ('h', 15), ('Q', 3), ('Q(x)', 1)]), 11: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1), ('Q', 1)]), 12: OrderedDict([('measure', 24), ('h', 

In [ ]:
eval_num(results.operation_counts)

119